---

## Theory

---

Let's learn how PINNs work! In this notebook, we'll implement a PINN following one of the [original papers](https://arxiv.org/pdf/1711.10561). This [video](https://www.youtube.com/watch?v=IDIv92Z6Qvc) was also helpful

The goal is to find a pde solution
looks cool: https://arxiv.org/pdf/2403.00599

https://github.com/idrl-lab/PINNpapers


The 1D viscous  Burgers' equation is

$$
\frac{\partial u}{\partial t} + u\frac{\partial u}{\partial x} = \nu \frac{\partial^2 u}{\partial x^2} \qquad x \in [-1, 1], \quad t \in [0, 1]
$$

where $u(x, t)$ gives the wave displacement and $\nu$ is the diffusion coefficient, describing

We will solve it with the Dirichlet boundary conditions 

$$
u(-1,t)=u(1,t)=0
$$

and initial conditions

$$
u(x,0) = - \sin(\pi x).
$$

First, we need to rewrite the PDE and its boundary / initial conditions into the form

\begin{align*}
\mathcal{D}[u(x, t)] &= f(x, t), \quad x \in \Omega \subset \mathbb{R}^d \\
B_k[u(x, t)] &= g_k(x, t), \quad x \in \Gamma_k \subset \partial \Omega \\
I_l[u(x, t=0)] &= h_l(x, t=0), \quad x \in \Omega \subset \mathbb{R}^d \\
\end{align*}

as described in [this notebook](./spring.ipynb). The PDE becomes


$$
\left( \frac{\partial }{\partial t} + u\frac{\partial }{\partial x} - \nu \frac{\partial^2 }{\partial x^2} \right) \big[u \big] - 0 = 0 
$$

The boundary conditions remain the same since they equal zero, but we will explicitly write them out. So the boundary / initial conditions are

\begin{align*}
1)& \quad u(-1,t) = 0 \quad \Rightarrow \quad u(-1,t) - 0 = 0 \\
2)& \quad u(1,t) = 0 \quad \Rightarrow \quad u(1,t) - 0 = 0 \\
3)& \quad u(x,t_\text{min}) = - \sin(\pi x) \quad \Rightarrow \quad u(x,t_\text{min}) + \sin(\pi x) = 0 \\

\end{align*}

The PINN is trained to directly approximate the solution to the differential equation

$$
N\!N(x, t;\theta) \approx u(x, t)
$$





---

### Building the loss function

The supervised **boundary loss** $ L_B(\theta)$ is given by
$$
L_B(\theta) = \sum_{k} \frac{\lambda_k}{N_{Bk}} \sum_{j}^{N_{Bk}} \left\| B_k \left[ NN(x_{kj}, t_j; \theta) \right] - g_k(x_{kj}, t_j) \right\|^2 
$$

where the outer summation with index $k$ is over the various given boundary conditions $B_k$ on the PDE. $N_{Bk}$ is the number of *boundary* training points $x_{kj}$ for boundary condition $B_k$. Then the inner summation over $j$ sums the $N_{Bk}$ squared residuals, and $\lambda_k$ is the regularization coefficient for the corresponding boundary condition.

The supervised **initial condition loss** $ L_I(\theta)$ is given by
$$
L_I(\theta) = \sum_{l} \frac{\lambda_l}{N_{Il}} \sum_{j}^{N_{Il}} \left\| I_l \left[ NN(x_{lj}, t_j; \theta) \right] - h_l(x_{lj}, t_j) \right\|^2 
$$

with the same idea as above except that now $t_j$ is really $t_\text{min}$, the initial time.

The unsupervised **physics loss** $ L_P(\theta)$ is given by 
$$
L_P(\theta) = \frac{1}{N_P} \sum_{i}^{N_P} \left\| \mathcal{D} \left[ NN(x_i, t_i; \theta) \right] - f(x_i, t_i) \right\|^2 
$$

where we sum the $N_P$ squared residuals.

Then our regularized loss function is made up of the physics loss, $L_P(\theta)$, the boundary loss, $L_B(\theta)$, and the initial condition loss, $L_I(\theta)$, with $t_\text{min} = 0$


\begin{align*}
\mathcal{L}(\theta) &= \lambda_P L_P(\theta) + \sum_k \lambda_{Bk} L_{Bk}(\theta) + \lambda_I L_I(\theta) \\
&= \frac{\lambda_P}{N_p} \sum_{i=1}^{N_P} \left( \left[\frac{\partial }{\partial t} + N\!N(x_i, t_i; \theta) \frac{\partial }{\partial x} - \nu \frac{\partial^2 }{\partial x^2} \right] N\!N (x_i, t_i; \theta)\right)^2 \\
&+ \frac{\lambda_{B1}}{N_{B1}} \sum_{j=1}^{N_{B1}} \left( N\!N(-1, t_j; \theta) - 0 \right)^2 + \frac{\lambda_{B2}}{N_{B2}} \sum_{j=1}^{N_{B2}} \left(N\!N(1, t_j; \theta) - 0\right)^2 \\
&+ \frac{\lambda_I}{N_I} \sum_{k=1}^{N_I} \left( N\!N(x_k, 0; \theta) + \sin(\pi x_k) \right)^2
\end{align*}

which we can simplify to

\begin{align*}
\mathcal{L}(\theta) &= \lambda_P L_P(\theta) + \sum \lambda_B L_B(\theta) + \lambda_I L_I(\theta) \\
&= \frac{\lambda_P}{N_p} \left\| \left[\frac{\partial }{\partial t} + N\!N(\vec{x}_P, \vec{t}_P; \theta) \frac{\partial }{\partial x} - \nu \frac{\partial^2 }{\partial x^2} \right] N\!N (\vec{x}_P, \vec{t}_P; \theta) \right\|^2\\
&+ \frac{\lambda_{B1}}{N_{B1}} \left\| N\!N(-1, \vec{t}_{B1}; \theta) \right\|^2 + \frac{\lambda_{B2}}{N_{B2}} \left\|N\!N(1, \vec{t}_{B2}; \theta) \right\|^2 \\
&+ \frac{\lambda_I}{N_I} \left\| N\!N(\vec{x}_I, 0; \theta) + \sin(\pi \vec{x}_I) \right\|^2
\end{align*}

where 
- $\vec{x}_P$ and $\vec{t}_P$ belong to the set of collocation points $\{(x, t)_i\}_{i=1}^{N_P}$
- $\vec{t}_{Bk}$ belongs to the set of boundary condition points $\{(g_k, t)_i\}_{i=1}^{N_{Bk}}$
- $\vec{x}_I$ belongs to the set of initial condition points $\{(x, t_\text{min})_i\}_{i=1}^{N_I}$

---

### Adding a transform layer

It looks like we have a few regularization hyperparameters to choose... one way we can remove some of these is by adding a transform function after the output layer of the network. This transform layer would enforce the initial condition at $t_\text{min}$ and look something like

```py
def transform_output(x, t, u, Tmin, h):
    return h(x, t) + u(x, t) * (t - Tmin) * (1 - x**2)
```

where `x` and `t` are the inputs to the PINN, `u` is the PINN output approximation of the PDE, `Tmin` is the initial condition time, and `h` is the initial condition function itself. As an equation,

$$
u_{\text{transformed}}(x, t) = h(x, t) + NN(x, t; \theta) \cdot (t - t_{\text{min}}) \cdot (1 - x^2)
$$

where, again, we have the PDE solution approximation $N\!N(x, t;\theta) \approx u(x, t)$. The idea behind this is simple: the transformation ensures that our initial condition function $h(x, t)$ is satisfied when $t = t_\text{min}$. In all other cases, when $t > t_\text{min}$, we see that the output is a combination of the specified initial condition function and the untransformed network output. In addition to enforcing the initial condition, it also satisfies the boundary conditions by making the output $0$ when $x = \pm 1$ for our particular initial condition $h(x, t) = -\sin(\pi x)$.

So by using this transform layer, we get to remove the initial condition loss term from our overall loss function -- simplifying it and giving us one less hyperparameter to tune :)

However, the transform layer has no learnable parameters and is technically not part of the neural network. But since it sits at the end of the network, it will have a strong influence on its output and, thus, the loss. This means that errors backpropagating through the network will tune its weights with information about the initial conditions recieved not explicitly from the loss function, but from the transform layer itself.

If we consider the transform layer as part of the network, we can write our new PDE solution approximation as
$$
N\!N'(x, t;\theta) \approx u_{\text{transformed}}(x, t, NN(x, t; \theta))
$$

where $N\!N'$ is the output of the network *with* the transform layer, and $N\!N$ is the previous vanilla output (included explicitly in the transformed solution here).

Then our new loss function becomes

\begin{align*}
\mathcal{L}(\theta) &= \lambda_P L_P(\theta) + \sum \lambda_B L_B(\theta)  \\
&= \frac{\lambda_P}{N_p} \left\| \left[\frac{\partial }{\partial t} + N\!N'(x_i, t_i; \theta) \frac{\partial }{\partial x} - \nu \frac{\partial^2 }{\partial x^2} \right] N\!N' (x_i, t_i; \theta) \right\|^2\\
&+ \frac{\lambda_{B1}}{N_{B1}} \left\| N\!N'(-1, t_j; \theta) \right\|^2 + \frac{\lambda_{B2}}{N_{B2}} \left\|N\!N'(1, t_j; \theta) \right\|^2 
\end{align*}

with final layer
$$
NN'(x, t; \theta) = -\sin(\pi x) + NN(x, t; \theta) \cdot (t - t_{\text{min}}) \cdot (1 - x^2)
$$


---

## Finally getting to some code!

---


In [35]:
import torch
import torch.nn as nn
from torchsummary import summary
from typing import Callable, List, Union

In [36]:
if torch.cuda.is_available():
    device = torch.device("cuda")
# elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
#     device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")

Using device: cpu


We're mainly going to be storing sets of $(x, t)$ tuples as $N \times 2$ tensors `xt`, where each row represents a sample, `xt[:, 0]` are the $x$ values, and `xt[:, 1]` are the $t$ values.

When indexing a column `i` in a tensor `a`, we can use `a[:, i:i+1]` instead of `a[:, i]` to preserve the column shape, if we so desire. 

In [37]:
a = torch.arange(10).reshape(5, 2)
print(a)
print(a[:, 1])
print(a[:, 1:2])

tensor([[0, 1],
        [2, 3],
        [4, 5],
        [6, 7],
        [8, 9]])
tensor([1, 3, 5, 7, 9])
tensor([[1],
        [3],
        [5],
        [7],
        [9]])


---

### Defining the network

First, we need a way of representing the boundary and intial conditions for our PDE.

In [38]:
# TODO: maybe store interval endpoints as well
# TODO: derivatives... see spring.ipynb for some ideas on this
class BC():
    """Boundary condition"""
    def __init__(self, x: float, f: Callable):
        self.x = x    # boundary location
        self.f = f    # boundary condition function

class IC():
    """Initial condition"""
    def __init__(self, Tmin: float, f: Callable):
        self.Tmin = Tmin  # initial time, usually just 0
        self.f = f        # initial condition function

# lhs and rhs have form lhs = lambda x, t: ...
class PDE():
    """PDE equation for the physics loss"""
    def __init__(self, LHS: Callable, RHS: Callable):
        self.LHS = LHS
        self.RHS = RHS

    def __call__(self, x: torch.Tensor, t: torch.Tensor) -> torch.Tensor:
        return self.LHS(x, t) - self.RHS(x, t)


Next we create our `transform` function to append to the output of the network, along with the initial condition `h`.

In [39]:
def old_transform(
        x: torch.Tensor, 
        t: torch.Tensor, 
        u: torch.Tensor, 
        Tmin: float, 
        h: Callable
        ) -> torch.Tensor:
    """Transform layer enforcing the initial condition"""

    def h(x: torch.Tensor, t: torch.Tensor) -> torch.Tensor:
        """Initial condition"""
        return -torch.sin(torch.pi * x) 
    
    ones = torch.ones_like(x)
    ts = t * torch.ones_like(t)
    return h(x, t) + u * (ts - Tmin) * (ones - x**2)


def transform(xt: torch.Tensor, u: torch.Tensor, ic: IC) -> torch.Tensor:
    """Transform layer enforcing the initial condition"""
    # get the x and t inputs to the network
    x = xt[:, 0:1]
    t = xt[:, 1:2]
    ones = torch.ones_like(x)
    ts = t * torch.ones_like(t)
    Tmins = ic.Tmin * torch.ones_like(t)
    return ic.f(x, t) + u * (ts - Tmins) * (ones - x**2)

We're now ready to define our `PINN`! We will initialize our weights using [Glorot / Xavier initialization](https://mmuratarat.github.io/2019-02-25/xavier-glorot-he-weight-init), and allow for the use of a custom, optional `transform_layer`.


In [40]:
class PINN(nn.Module):
    """PINN model for specifically solving Burgers' equation"""
    def __init__(
            self,
            layers: List[int],
            bcs: List[BC],
            ics: List[IC],
            activation: Callable = nn.Tanh,
            transform_layer: Callable = None,
            loss_fn: Callable = nn.MSELoss
            ):
        super(PINN, self).__init__()
        self.bcs = bcs                     
        self.ics = ics                     
        self.activation = activation()     # TODO: i don't like doing act() here, see if i can do act
        self.transform_layer = transform_layer
        self.loss_fn = loss_fn
        
        # define the MLP layers and initialize the weights
        self.fc = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        self._xavier_init()

    def _xavier_init(self):
        for layer in self.fc:
            nn.init.xavier_normal_(layer.weight)
            nn.init.zeros_(layer.bias)

    def forward(self, xt: torch.Tensor) -> torch.Tensor:
        # only save input if transform is provided
        if self.transform_layer is not None:
            input = xt.clone().detach().requires_grad_(True)  # TODO: check this later
        
        for layer in self.fc:
            xt = self.activation(layer(xt))

        # transform output if provided
        if self.transform_layer is not None:
            # the input to the network was xt, and the output is now xt... # TODO: better notation
            return self.transform_layer(xt=input, u=xt, ic=self.ics[0])
        return xt
    
    def physics_loss(self, xt: torch.Tensor, lam: float, nu: float) -> torch.Tensor:
        """Get the loss dictated by the PDE"""
        # get the networks predictions
        u = self.forward(xt)

        # get the gradients: x = xt[:, 0:1], t = xt[:, 1:2]
        ut = torch.autograd.grad(u, xt[:, 1:2], torch.ones_like(u), create_graph=True)[0]
        ux = torch.autograd.grad(u, xt[:, 0:1], torch.ones_like(u), create_graph=True)[0]
        uxx = torch.autograd.grad(ux, xt[:, 1:2], torch.ones_like(ux), create_graph=True)[0]
        
        # put the terms into the PDE and get the residual
        residual = lam * (ut + u*ux - nu*uxx)
        return torch.mean(residual**2)
        # return self.loss_fn(lam * (ut + u*ux - nu*uxx), torch.zeros_like(u))
    

    # TODO: the ic and bc losses are really the same template -- combine these into one function later
    # TODO: also annoying to generate x's or t's for the losses each time
    #       - could maybe index sample points with (x_bc, t) and (x, t_ic) and pass those in instead
    def bc_loss(self, t: torch.Tensor) -> torch.Tensor:
        """Get the loss for the boundary conditions -- xs and us are lists of x and u values for the BCs"""
        loss = 0
        for bc in self.bcs:
            x = bc.x * torch.ones_like(t, requires_grad=True) # make x the same size as t
            xt = torch.cat((x, t), dim=1)                     # concat along dim=1 to make the x and t columns of xt
            u = self.forward(xt)                              # get the networks prediction
            targets = bc.f(x, t)                              # get true BC value
            loss += self.loss_fn(u, targets)                  # add the loss to the total
        return loss
    
    def ic_loss(self, x: torch.Tensor) -> torch.Tensor:
        """Get the loss for the initial conditions"""
        loss = 0
        for ic in self.ics:
            t = ic.Tmin * torch.ones_like(x, requires_grad=True) # make t the same size as x
            xt = torch.cat((x, t), dim=1)                     # concat along dim=1 to make the x and t columns of xt
            u = self.forward(xt)                              # get the networks prediction 
            targets = ic.f(x, t)                              # get true IC value
            loss += self.loss_fn(u, targets)                  # add the loss to the total
        return loss

Let's set our boundary and initial conditions. Recall, we had

\begin{equation*}
\begin{cases}
\text{BC1}: & u(-1, t) = 0, \\
\text{BC2}: & u(1, t) = 0, \\
\text{IC1}: & u(x, t_{\min}) = -\sin(\pi x)
\end{cases}
\end{equation*}


In [41]:
bc1 = BC(x=-1, f=lambda x, t: 0)
bc2 = BC(x=1, f=lambda x, t: 0)
bcs = [bc1, bc2]

ic = IC(Tmin=0, f=lambda x, t: -torch.sin(torch.pi * x))
ics = [ic]

Now we can define our PDE. 

$$
\frac{\partial u}{\partial t} + u\frac{\partial u}{\partial x} = \nu \frac{\partial^2 u}{\partial x^2} \qquad x \in [-1, 1], \quad t \in [0, 1]
$$

> Currently have no framework for representing derivatives outside of the network, so this part is not used

In [ ]:
LHS = lambda x, t: 0
RHS = lambda x, t: 0
pde = PDE(LHS, RHS)

Now we can instantiate our network. The input layer should accecpt tensors of shape $(2, )$, as we'll be passing in tuples $(x, t)$. The output layer should only contain a single neuron, since the network output serves as the solution to the PDE, $u(x, t)$.

The optimizer ... LBFGS..

In [42]:
# pinn = PINN([2, 20, 20, 20, 1], transform)
pinn = PINN([2, 20, 20, 20, 1], bcs=bcs, ics=ics).to(device)
opt = torch.optim.Adam(pinn.parameters(), lr=0.001)
print(f"Number of parameters: {sum(p.numel() for p in pinn.parameters())}")

Number of parameters: 921


We can print a summary of the model by passing in a mock tensor of size $(N, 2)$, where $N$ is the number of samples being processed in parallel.

In [43]:
summary(pinn, input_size=(10, 2))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 10, 20]              60
              Tanh-2               [-1, 10, 20]               0
            Linear-3               [-1, 10, 20]             420
              Tanh-4               [-1, 10, 20]               0
            Linear-5               [-1, 10, 20]             420
              Tanh-6               [-1, 10, 20]               0
            Linear-7                [-1, 10, 1]              21
              Tanh-8                [-1, 10, 1]               0
Total params: 921
Trainable params: 921
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.00
Estimated Total Size (MB): 0.01
----------------------------------------------------------------


Next we create our supervised initial & boundary points, and unsupervised physics / collocation points.

In [56]:
# define boundary points, for the boundary loss
bc1_t = torch.linspace(0, 1, 10).view(-1, 1).requires_grad_(True).to(device)
bc2_t = torch.linspace(0, 1, 10).view(-1, 1).requires_grad_(True).to(device)

# t_boundary = torch.Tensor(0.).view(-1,1).requires_grad_(True).to(device)# (1, 1)


# define initial condition points, for the initial condition loss
ic_x = torch.linspace(-1, 1, 10).view(-1, 1).requires_grad_(True).to(device)

# define collocation points over the entire domain, for the physics loss
col_x = torch.rand(10, 1).requires_grad_(True).to(device)
# t_physics = torch.linspace(0,1,30).view(-1,1).requires_grad_(True).to(device)# (30, 1)


In [ ]:
# TODO: figure out beset way to setup grid
# generate collocation points
x_collocation = torch.linspace(-1, 1, 1000, device=device)
t_collocation = torch.linspace(0, 1, 1000, device=device)
x_collocation, t_collocation = torch.meshgrid(x_collocation, t_collocation, indexing="ij")
x_collocation = x_collocation.flatten().unsqueeze(1).to(device)
t_collocation = t_collocation.flatten().unsqueeze(1).to(device)

# generate boundary points
t_boundary = torch.linspace(0, 1, 1000, device=device).unsqueeze(1).to(device)

---

### Training the network

Next we can write the training loop.

In [44]:
def optimization_step(pinn, opt, x, t, tmin):
    opt.zero_grad()
    input = torch.cat([x, t], dim=1)
    output = pinn(input)
    loss = pinn.ic_loss(x) + pinn.bc_loss(t) + pinn.physics_loss(input, 1, 0.01)
    loss.backward()
    opt.step()
    return loss

First, let's find a good learning rate

In [45]:
# TODO: figure out how to use with model
g = torch.Generator().manual_seed(2147483647)  # rng seed


In [46]:
num_epochs = 10000

In [ ]:
# 1000 exponents evenly spaced between -3 and 0
lre = torch.linspace(-3, 0, steps=1000)
lrs = 10**lre

In [47]:
# find the minimum loss and the corresponding learning rate
min_loss = min(lossi)
min_loss_index = lossi.index(min_loss)
min_lrei = lrei[min_loss_index]

min_loss = min(lossi)
min_loss_index = lossi.index(min_loss)
min_lr = lrs[min_loss_index]
print(f"Min learning rate exponent: {min_lrei:.2f} and corresponding learing rate: {10**min_lrei:.2e}")

NameError: name 'lossi' is not defined

In [ ]:
# now we can plot the results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# loss vs learning rate exponent
ax1.plot(lrei, lossi)
ax1.axvline(x=min_lrei, color="r", linestyle="--", label=f"Min loss at {min_lrei:.2f}")
ax1.set_xlabel("Learning Rate Exponent")
ax1.set_ylabel("Loss")
ax1.set_title("Loss vs Learning Rate Exponent")
ax1.legend()

# loss vs learning rate
ax2.plot(lrs, lossi)
ax2.axvline(x=min_lr, color="r", linestyle="--", label=f"Min loss at {min_lr:.2e}")
ax2.set_xlabel("Learning Rate")
ax2.set_ylabel("Loss")
ax2.set_title("Loss vs Learning Rate")
ax2.legend()

plt.tight_layout()
plt.show()


---

### Plotting the results